# Hough Circle Transform Segmentation /18/01/2021

1.  import the libraries you need 

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import measure
import glob
from scipy.io import loadmat
from scipy.spatial import distance
from scipy import interpolate
from matplotlib import pyplot as plt
import time

2.   mount GoogleDrive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

3.read the image from Drive

In [ ]:
img_gray = cv2.imread("/content/gdrive/MyDrive/mono2.png", cv2.IMREAD_GRAYSCALE)

4. import reference map for frame 1

In [ ]:
labels = np.loadtxt("/content/gdrive/MyDrive/labels.txt", dtype=float)
calibMaskold = np.zeros((172, 2), dtype=float)
calibMaskold[:, 0] = np.loadtxt("/content/gdrive/MyDrive/centrex.txt", dtype=float)
calibMaskold[:, 1] = np.loadtxt("/content/gdrive/MyDrive/centrey.txt", dtype=float)
spotDiam = np.loadtxt("/content/gdrive/MyDrive/centreradi.txt", dtype=float)
meanDiam = np.mean(spotDiam)

5. define function for image brightness control

In [ ]:
begin = time.time()
def increase_brightness(img, value=30):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img


6. apply blur and morphological operations:


In [ ]:
img = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img = cv2.GaussianBlur(img, (5, 5), 0)
cimg = cv2.medianBlur(img, 5)
rows = img.shape[0]
kernel = np.ones((3, 3), np.uint8)
img = cv2.dilate(cimg, kernel, iterations=2)

7. detect circles with the Hough Circle Transform function

In [ ]:

circles = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT, 1, 30, param1=5, param2=20, minRadius=10, maxRadius=23) # rows/24
circles = np.uint16(np.around(circles))


centers = []
radi = []

for i in circles[0,:]:
	# Get circle centre position
    center = (i[0], i[1])
    centers.append(center)
    # Draw centre of the detected circle on input image
    cv2.circle(img, center, 1, (0, 100, 100), 3)
    # Get circle radius
    radius = i[2]
    radi.append(radius)
    # Draw perimeter of detected cirle on input image
    cv2.circle(img, center, radius, (255, 0, 255), 3)

centers = np.array(centers)
radi = np.array(radi)
nondetected = 172 - len(radi) # the spots that have not been detected


8.-optional- correct the labels based on the weighted centre of the detected spots

In [ ]:
#find the weighted centre of the segmented
wc1sum = [sum(x) for x in zip(*centers)]

wc1 = np.zeros((2, 1), dtype=float)
for i, element in enumerate(wc1sum):
 wc1[i] = element / len(radi)

#find the weighted centre of the labels
wc2sum = [sum(x) for x in zip(*calibMaskold)]

wc2 = np.zeros((2, 1), dtype=float)
for i, element in enumerate(wc2sum):
 wc2[i] = element / len(calibMaskold)

#find the weighted centre differences
dwc21= np.zeros((2, 1), dtype=float)
dwc21[0] = wc1[0]-wc2[0]
dwc21[1] = wc1[1]-wc2[1]

#correct the labels-optional-
calibMask = np.zeros((172, 2), dtype=float)
calibMask[:, 0] = calibMaskold[:, 0] + dwc21[0]
calibMask[:, 1] = calibMaskold[:, 1] + dwc21[1]

9. Match the segmented spots with the reference spots labels-3 per segmented spot here-maybe we will change this to keep only the closest and not the three closest

In [ ]:
nondetectedlabels = [6, 7, 19, 36, 107, 112, 136, 138, 139, 153, 154, 155, 157, 161]
NumList = []
idofspot = []
matchedspots = []
matcheddistances = []

distanceandid = np.zeros((172, 2), dtype=int)
distances2keep = 3


list_1d_labels = []
list_2d_x = []
list_2d_y = []
list_2d_r = []
list_2d_d = []
for i in range(len(calibMask)):
    list_1d_labels.append(i + 1)
    distances_to_i = []
    for j in range(len(centers)):
        dists = distance.euclidean(centers[j], calibMask[i])
        distances_to_i.append(dists)
    sorted_ind = np.argsort(distances_to_i)
    list_x = []
    list_y = []
    list_r = []
    list_d = []
    for k in range(distances2keep):
        ind_tmp = sorted_ind[k]
        x_tmp, y_tmp = centers[ind_tmp]
        list_x.append(x_tmp)
        list_y.append(y_tmp)
        list_r.append(radi[ind_tmp])
        list_d.append(distances_to_i[ind_tmp])
    list_2d_x.append(list_x)
    list_2d_y.append(list_y)
    list_2d_r.append(list_r)
    list_2d_d.append(list_d)

10. keep the closest label

In [ ]:
closestd = []
closestx = []
closesty = []
closestr = []
closestlabels = []

for i in range(0, len(list_1d_labels)):#for every label
  label_tmp = i+1
  if not label_tmp in nondetectedlabels:
                distancematch = list_2d_d[i][0]
                centrexmatch = list_2d_x[i][0]
                centreymatch = list_2d_y[i][0]
                radiusmatch = list_2d_r[i][0]
                labelmatch = i+1
                closestd.append(distancematch)
                closestx.append(centrexmatch)
                closesty.append(centreymatch)
                closestr.append(radiusmatch)
                closestlabels.append(labelmatch)
                end = time.time()
print(f"Total runtime (seconds) of the program is {end - begin}")